## Non-dominated Sorting Genetic Algorithm Implementation

Developed by Daniel Frutos Rodriguez for _"A Comparison of Optimization Techniques for Solving the Team Formation Problem"_.

Frutos-Rodriguez D., Barrios-Fleitas Y., and Lalla E. 2025. _A Comparison of Optimization Techniques for Solving the Team Formation Problem._ In _Proceedings Placeholder_. ACM, New York, NY, USA, 6 pages. [https://doi.org/10.1145/nnnnnnn.nnnnnnn](https://doi.org/10.1145/nnnnnnn.nnnnnnn)

### Libraries

In [2]:
import pandas as pd # For datasets
import utils.fitness_functions as ff
import utils.restriction_checker as rc
import models.team_assignment as ta
import random # For randomness
import os # For output
from contextlib import redirect_stdout # For output
import datetime # For output
from copy import deepcopy # For crossover function
import math # For ceiling function for team partitions

#### Constants

In [3]:
# Data

PROJECTS = ['Shotmaniacs', 'actFact', 'Honours Programme', 'Voice', 'Topicus', 'Earnit', 'Inter-actief']  # Possible projects.
DATA = 'data/dataset_small.csv'  # Dataset reference.
DATASET = pd.read_csv(DATA)  # Dataset stored as a DataFrame.

# Safeguards

MAX_CROSSOVER_ATTEMPTS = 10  # Maximum number of attempts in crossover algorithm (avoiding infinite loop).
MAX_MUTATION_ATTEMPTS = 10  # Maximum number of attempts in mutation algorithm (avoiding infinite loop).

# Experimental controls

NUMBER_OF_GENERATIONS = 100  # Number of generations to iterate in the NSGA-II loop.
OFFSPRING_PER_GENERATION = 8  # Number of offspring generated in each generation.
POPULATION_SIZE = 10  # Number of individuals in the population.

# Output constants

RUN_TIME = datetime.datetime.now().strftime("run_%Y%m%d_%H%M%S")  # Constant for output folder naming.
BASE_DIR = "output/nsga-ii"  # Base directory for outputs.

#### Output Configuration

In [4]:
def save_arrangements_batch(arrangements, iteration, run_time=RUN_TIME):
    # Create base and run-time output folder if not already created
    os.makedirs(BASE_DIR, exist_ok=True)
    run_folder = os.path.join(BASE_DIR, f"{run_time}", f"generation_{iteration}")
    os.makedirs(run_folder, exist_ok=True)

    diversity_scores = []
    satisfaction_scores = []

    for i, arrangement in enumerate(arrangements, start=1):
        filename = os.path.join(run_folder, f"assignment_{i}.txt")

        with open(filename, "w") as f:
            with redirect_stdout(f):
                diversity, satisfaction = ff.evaluate_objectives_separately(arrangement)
                print(f"NSGA-II Generation {iteration} - Solution {i}")
                print(f"Diversity Score: {diversity:.4f}")
                print(f"Satisfaction Score: {satisfaction:.4f}")

        diversity_scores.append(diversity)
        satisfaction_scores.append(satisfaction)

    return diversity_scores, satisfaction_scores

def log_nsga_pareto_per_generation(iteration, diversity_scores, satisfaction_scores, performance_log, arrangements_computed, decimals=4,):
    
    pareto_map = {}

    for div, sat in zip(diversity_scores, satisfaction_scores):
        sat_rounded = round(sat, decimals)
        if sat_rounded not in pareto_map or div > pareto_map[sat_rounded]:
            pareto_map[sat_rounded] = div

    # Sort keys (satisfaction levels)
    sorted_sats = sorted(pareto_map.keys())

    with open(performance_log, "a") as f:
        f.write(f"\nGeneration {iteration} Pareto Front (best diversity for each satisfaction):\n")
        for sat in sorted_sats:
            f.write(f"  Satisfaction: {sat:.{decimals}f}, Best Diversity: {pareto_map[sat]:.{decimals}f}\n")
        f.write(f"Total Computations: [{arrangements_computed}]")


#### Initial Population Generation

In [5]:
def create_random_teams(df, min_size=5, max_size=6, project_pool=PROJECTS):
    students = df.to_dict(orient='records')
    random.shuffle(students)

    total_students = len(students)
    remainder = total_students % min_size
    num_six_person_teams = remainder
    num_five_person_teams = (total_students - (max_size * num_six_person_teams)) // min_size
    total_teams = num_five_person_teams + num_six_person_teams

    def assign_projects_to_teams(num_teams, projects):
        base = num_teams // len(projects)
        extra = num_teams % len(projects)
        project_assignments = []
        for idx, project in enumerate(projects):
            count = base + (1 if idx < extra else 0)
            project_assignments.extend([project] * count)
        random.shuffle(project_assignments)
        return project_assignments

    project_assignments = assign_projects_to_teams(total_teams, project_pool)
    project_counters = {proj: 0 for proj in project_pool}

    teams = []

    for _ in range(num_six_person_teams):
        team = []
        tcs_count = 0
        nationality_counts = {}

        while len(team) < max_size:
            valid_candidates = [
                s for s in students
                if ('TCS' not in s['Program'] or tcs_count < 4)
                and (s['Nationality'] == 'Dutch' or nationality_counts.get(s['Nationality'], 0) < 3)
            ]

            selected = random.choice(valid_candidates)
            team.append(selected)
            students.remove(selected)

            if 'TCS' in selected['Program']:
                tcs_count += 1
            if selected['Nationality'] != 'Dutch':
                nationality_counts[selected['Nationality']] = nationality_counts.get(selected['Nationality'], 0) + 1

        project = project_assignments[len(teams)]
        project_counters[project] += 1
        team_id = f"{project} {project_counters[project]}"
        teams.append(ta.TeamAssignment(team_id, team, project, fitness=0.0))

    for _ in range(num_five_person_teams):
        team = []
        tcs_count = 0
        nationality_counts = {}

        while len(team) < min_size:
            valid_candidates = [
                s for s in students
                if ('TCS' not in s['Program'] or tcs_count < 4)
                and (s['Nationality'] == 'Dutch' or nationality_counts.get(s['Nationality'], 0) < 3)
            ]
            if not valid_candidates:
                return None

            selected = random.choice(valid_candidates)
            team.append(selected)
            students.remove(selected)

            if 'TCS' in selected['Program']:
                tcs_count += 1
            if selected['Nationality'] != 'Dutch':
                nationality_counts[selected['Nationality']] = nationality_counts.get(selected['Nationality'], 0) + 1

        project = project_assignments[len(teams)]
        project_counters[project] += 1
        team_id = f"{project} {project_counters[project]}"
        teams.append(ta.TeamAssignment(team_id, team, project, fitness=0.0))

    return teams if rc.is_valid_arrangement(teams, total_students, project_pool) else None


# Alternative method for creating random teams as a subset of the crossover function, ensuring restrictions are met for the whole dataset and not just a subset.
def create_random_teams_for_subset(df, current_project_counters, max_per_project, min_size=5, max_size=6):
    students = df.to_dict(orient='records')
    random.shuffle(students)

    total_students = len(students)
    remainder = total_students % min_size
    num_six_person_teams = remainder
    num_five_person_teams = (total_students - (max_size * num_six_person_teams))

    teams = []

    # Build the teams
    for _ in range(num_six_person_teams):
        team = []
        tcs_count = 0
        nationality_counts = {}
        while len(team) < max_size:
            valid_candidates = [
                s for s in students
                if ('TCS' not in s['Program'] or tcs_count < 4)
                and (s['Nationality'] == 'Dutch' or nationality_counts.get(s['Nationality'], 0) < 3)
            ]
            if not valid_candidates:
                return None
            selected = random.choice(valid_candidates)
            team.append(selected)
            students.remove(selected)
            if 'TCS' in selected['Program']:
                tcs_count += 1
            if selected['Nationality'] != 'Dutch':
                nationality_counts[selected['Nationality']] = nationality_counts.get(selected['Nationality'], 0) + 1

        available_projects = [proj for proj, count in current_project_counters.items() if count < max_per_project]
        if not available_projects:
            return None
        project = random.choice(available_projects)
        current_project_counters[project] += 1
        team_id = f"{project} {current_project_counters[project]}"
        teams.append(ta.TeamAssignment(team_id, team, project, fitness=0.0))

    for _ in range(num_five_person_teams):
        team = []
        tcs_count = 0
        nationality_counts = {}
        while len(team) < min_size:
            valid_candidates = [
                s for s in students
                if ('TCS' not in s['Program'] or tcs_count < 4)
                and (s['Nationality'] == 'Dutch' or nationality_counts.get(s['Nationality'], 0) < 3)
            ]
            if not valid_candidates:
                return None
            selected = random.choice(valid_candidates)
            team.append(selected)
            students.remove(selected)
            if 'TCS' in selected['Program']:
                tcs_count += 1
            if selected['Nationality'] != 'Dutch':
                nationality_counts[selected['Nationality']] = nationality_counts.get(selected['Nationality'], 0) + 1

        available_projects = [proj for proj, count in current_project_counters.items() if count < max_per_project]
        if not available_projects:
            return None
        project = random.choice(available_projects)
        current_project_counters[project] += 1
        team_id = f"{project} {current_project_counters[project]}"
        teams.append(ta.TeamAssignment(team_id, team, project, fitness=0.0))

    return teams

    

#### Domination Function

In [6]:
def dominates(a, b):
    return (a[0] >= b[0] and a[1] >= b[1]) and (a[0] > b[0] or a[1] > b[1])

#### Non-dominated Sorting

In [7]:
def fast_non_dominated_sort(pop_objs):
    S = [[] for _ in range(len(pop_objs))]
    n = [0] * len(pop_objs)
    rank = [0] * len(pop_objs)
    fronts = [[]]
    for p in range(len(pop_objs)):
        for q in range(len(pop_objs)):
            if dominates(pop_objs[p], pop_objs[q]):
                S[p].append(q)
            elif dominates(pop_objs[q], pop_objs[p]):
                n[p] += 1
        if n[p] == 0:
            rank[p] = 0
            fronts[0].append(p)
    i = 0
    while fronts[i]:
        next_front = []
        for p in fronts[i]:
            for q in S[p]:
                n[q] -= 1
                if n[q] == 0:
                    rank[q] = i + 1
                    next_front.append(q)
        i += 1
        fronts.append(next_front)
    return fronts[:-1], rank

#### Crowding Distance Sorting

In [8]:
def crowding_distance(front, pop_objs):
    distances = [0.0] * len(front)
    for m in range(2):
        values = [pop_objs[i][m] for i in front]
        sorted_indices = sorted(range(len(values)), key=lambda x: values[x])
        distances[sorted_indices[0]] = distances[sorted_indices[-1]] = float('inf')
        for i in range(1, len(front) - 1):
            prev_val = values[sorted_indices[i - 1]]
            next_val = values[sorted_indices[i + 1]]
            distances[sorted_indices[i]] += (next_val - prev_val) / (max(values) - min(values) + 1e-9)
    return distances

#### Selection

In [9]:
def select_population(population, pop_objs):
    fronts, _ = fast_non_dominated_sort(pop_objs)
    new_population = []
    for front in fronts:
        if len(new_population) + len(front) > POPULATION_SIZE:
            distances = crowding_distance(front, pop_objs)
            sorted_front = sorted(zip(front, distances), key=lambda x: x[1], reverse=True)
            new_population += [population[i] for i, _ in sorted_front[:POPULATION_SIZE - len(new_population)]]
            break
        else:
            new_population += [population[i] for i in front]
    return new_population

#### Crossover

In [10]:
def crossover(parent1, parent2, attempt_counter):
    crossover_attempts = 0
    while (crossover_attempts < MAX_CROSSOVER_ATTEMPTS):
        attempt_counter[0] += 1
        selected_teams = random.sample(parent1, k=random.randint(1, len(parent1) - 1))
        
        # Extract IDs from selected teams' members
        used_ids = set(member['ID'] for team in selected_teams for member in team.members)

        # Start building child from selected teams
        child_teams = deepcopy(selected_teams)

        # Add non-overlapping teams from parent2
        for team in parent2:
            team_ids = [member['ID'] for member in team.members]
            if any(id in used_ids for id in team_ids):
                continue
            child_teams.append(deepcopy(team))
            used_ids.update(team_ids)

        # Determine max_per_project for the full dataset
        total_teams_full = len(DATASET) // 5  # or ceil if needed
        max_per_project = math.ceil(total_teams_full / len(PROJECTS))

        # Compute used project counts so far
        from collections import Counter
        used_project_counts = Counter(team.project for team in child_teams)

        # Fill remaining students while respecting project limits
        all_ids = DATASET['ID'].tolist()
        remaining_ids = [id for id in all_ids if id not in used_ids]
        df_by_id = DATASET.set_index('ID').to_dict(orient='index')
        remaining_students = [dict(df_by_id[id], ID=id) for id in remaining_ids]

        if remaining_students:
            remaining_df = pd.DataFrame(remaining_students)
            new_teams = create_random_teams_for_subset(
                remaining_df, 
                current_project_counters=used_project_counts, 
                max_per_project=max_per_project
            )
            if new_teams is None:
                crossover_attempts += 1
                continue
            child_teams += new_teams

        if rc.is_valid_arrangement(child_teams, total_students=len(DATASET), projects=PROJECTS):
            return child_teams
        
        crossover_attempts += 1

    return create_random_teams(DATASET) # Fallback. Random arrangement.

#### Mutation

In [11]:
def mutate(arrangement, attempt_counter):
    mutation_attempts = 0
    while mutation_attempts < MAX_MUTATION_ATTEMPTS:
        attempt_counter[0] += 1
        mutated = deepcopy(arrangement)

        if not isinstance(mutated, list):
            mutated = list(mutated)

        # Pick two distinct teams
        team_a, team_b = random.sample(mutated, 2)

        # Pick one random member from each
        student_a = random.choice(team_a.members)
        student_b = random.choice(team_b.members)

        # Swap them
        team_a.members.remove(student_a)
        team_b.members.remove(student_b)
        team_a.members.append(student_b)
        team_b.members.append(student_a)

        # Check if resulting arrangement is valid
        if rc.is_valid_arrangement(mutated, total_students=len(DATASET), projects=PROJECTS):
            return mutated
        
        mutation_attempts += 1

    return create_random_teams(DATASET) # Fallback. Random arrangement.

#### GA Execution

In [12]:
def execute_nsga_ii():

    # Setup performance log
    run_time = datetime.datetime.now().strftime("run_%Y%m%d_%H%M%S")
    performance_log = os.path.join(BASE_DIR, f"{run_time}", "performance.txt")
    os.makedirs(os.path.dirname(performance_log), exist_ok=True)
    with open(performance_log, "w") as f:
        f.write(f"Performance log for run: {run_time}\n")

    # Data for progress tracking
    attempted_arrangements = [0]
    pareto_log = {}  # generation → list of (diversity, satisfaction)
    computation_log = {} # generation → number of attempted arrangements

    # Initial population
    population = []
    while len(population) < 10:
        individual = create_random_teams(DATASET)
        if individual:
            population.append(individual)

    for generation in range(NUMBER_OF_GENERATIONS + 1):

        # Generate offspring
        offspring = []
        while len(offspring) < OFFSPRING_PER_GENERATION:
            if len(population) < 2:
                break  # prevent sampling error
            parents = random.sample(population, 2)
            child = crossover(parents[0], parents[1], attempted_arrangements)
            if child:
                mutated = mutate(child, attempted_arrangements)
                if mutated:
                    offspring.append(mutated)

        # Combine offspring + parent population
        combined_population = population + offspring

        # Evaluate and save all combined individuals
        diversity_scores, satisfaction_scores = save_arrangements_batch(combined_population, generation, run_time)

        # Run fast non-dominated sort
        combined_objs = list(zip(diversity_scores, satisfaction_scores))
        fronts, _ = fast_non_dominated_sort(combined_objs)

        # Select next generation
        selected_indices = fronts[0]  # Pareto front
        population = [combined_population[i] for i in selected_indices]
        pareto_objs = [combined_objs[i] for i in selected_indices]

        # Log current Pareto structure
        log_nsga_pareto_per_generation(
            iteration=generation,
            diversity_scores=[div for div, sat in pareto_objs],
            satisfaction_scores=[sat for div, sat in pareto_objs],
            performance_log=performance_log,
            arrangements_computed=attempted_arrangements[0],
        )

        # Store front for visualization
        pareto_log[generation] = pareto_objs

        # Store computations for visualization
        computation_log[generation] = attempted_arrangements[0]

    return pareto_log, computation_log
